# Imports and data processing

In [1]:
!pip install pretty_midi

     |████████████████████████████████| 5.6MB 11.0MB/s 
     |████████████████████████████████| 61kB 9.3MB/s 
  Created wheel for pretty-midi: filename=pretty_midi-0.2.9-cp37-none-any.whl size=5591954 sha256=e18a09f4c1c5f516aad49e7901e5d38d6c42523c5d4579a1b70e6df457d85cd0
  Stored in directory: /root/.cache/pip/wheels/4c/a1/c6/b5697841db1112c6e5866d75a6b6bf1bef73b874782556ba66
Successfully built pretty-midi


In [3]:
import random
import os
import pretty_midi
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns


import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.autograd import Variable
from torch.utils.data import TensorDataset

import itertools
import random as rd
import copy

# we use GPU if available, otherwise CPU
device_to_use = "cuda:0" if torch.cuda.is_available() else "cpu"
device = torch.device(device_to_use)
print(f"Using {device_to_use} device")


Using cuda:0 device


In [4]:
!unzip "classical3D.zip"

Archive:  classical3D.zip
   creating: content/classical3D_nice/
  inflating: content/classical3D_nice/classical_content_18.npy  
  inflating: content/classical3D_nice/classical_content_14.npy  
  inflating: content/classical3D_nice/classical_content_21.npy  
  inflating: content/classical3D_nice/classical_content_153.npy  
  inflating: content/classical3D_nice/classical_content_98.npy  
  inflating: content/classical3D_nice/classical_content_127.npy  
  inflating: content/classical3D_nice/classical_content_39.npy  
  inflating: content/classical3D_nice/classical_content_107.npy  
  inflating: content/classical3D_nice/classical_content_65.npy  
  inflating: content/classical3D_nice/classical_content_150.npy  
  inflating: content/classical3D_nice/classical_content_92.npy  
  inflating: content/classical3D_nice/classical_content_99.npy  
  inflating: content/classical3D_nice/classical_content_137.npy  
  inflating: content/classical3D_nice/classical_content_159.npy  
  inflating: conten

In [5]:
!unzip "jazz3D.zip"

Archive:  jazz3D.zip
   creating: content/jazz3D/
  inflating: content/jazz3D/jazz_content_131.npy  
  inflating: content/jazz3D/jazz_content_23.npy  
  inflating: content/jazz3D/jazz_content_176.npy  
  inflating: content/jazz3D/jazz_content_6.npy  
  inflating: content/jazz3D/jazz_content_227.npy  
  inflating: content/jazz3D/jazz_content_237.npy  
  inflating: content/jazz3D/jazz_content_89.npy  
  inflating: content/jazz3D/jazz_content_41.npy  
  inflating: content/jazz3D/jazz_content_218.npy  
  inflating: content/jazz3D/jazz_content_222.npy  
  inflating: content/jazz3D/jazz_content_257.npy  
  inflating: content/jazz3D/jazz_content_146.npy  
  inflating: content/jazz3D/jazz_content_55.npy  
  inflating: content/jazz3D/jazz_content_126.npy  
  inflating: content/jazz3D/jazz_content_239.npy  
  inflating: content/jazz3D/jazz_content_141.npy  
  inflating: content/jazz3D/jazz_content_69.npy  
  inflating: content/jazz3D/jazz_content_59.npy  
  inflating: content/jazz3D/jazz_content

In [6]:
def get_samples(style="J", length=64, n=1000, random=False, min_note_per_sample=100):
    """Return the preprocessed samples

    Parameters
    ----------
    style : str, optional
        Style of the music, one of {"J", "C"} (for "Jazz" and "Classic"), by default "J"
    length : int, optional
        The length in time steps of each samples
    n : int, optional
        Number of samples which will be returned, by default 100
    random : bool, optional
        Whether to randomize the samples or not, by default False

    Returns
    -------
    List (of Numpy arrays)
        List of samples
    """

    folder = ""
    if style == "J":
      folder = "content/jazz3D"
    
    elif style == "C":
      folder = "content/classical3D"

    else:
      raise Exception("'style' must be one of {'J', 'C'}")
   
      
    list_filenames = os.listdir(folder)
    nb_samples_per_file = max(int(n/len(list_filenames)), 1)

    samples = []
    for filename in list_filenames:
      if filename[-3:] =="npy":
         
          
        filepath = folder + "/" + filename
        

        all_file = np.load(filepath).astype(int)
        all_file_size = len(all_file)

        for k in range(nb_samples_per_file):
          first_index = rd.randint(0, int((all_file_size-length-1)/4))
          sample = all_file[4*first_index:4*first_index+length,:,0:2]
          while sample.sum() < min_note_per_sample:
            first_index = rd.randint(0, int((all_file_size-length-1)/4))
            sample = all_file[4*first_index:4*first_index+length,:,0:2]


          samples.append(sample.reshape((1, 64 , 84, 2)))
          
    return np.vstack(samples)




# Model creation

In [7]:
class ResidualBlock(nn.Module):
    def __init__(self, in_features):
        super(ResidualBlock, self).__init__()
        #### They will be used as 2x16x21

        #  conv_block = [  nn.ReflectionPad3d(1),
        #                 nn.Conv3d(in_features, in_features, (1,3,3)),
        #                 nn.InstanceNorm3d(in_features),
        #                 nn.ReLU(inplace=True),
        #                 nn.ReflectionPad3d(1),
        #                 nn.Conv3d(in_features, in_features, (1,3,3)),
        #                 nn.InstanceNorm3d(in_features)]


        
        conv_block = [  nn.Conv3d(in_features, in_features, (1,3,3), (1,1,1), padding=(0,1,1)),
                        nn.InstanceNorm3d(in_features),
                        nn.ReLU(inplace=True),
                        nn.Conv3d(in_features, in_features, (1,3,3), (1,1,1), padding=(0,1,1)),
                        nn.InstanceNorm3d(in_features)]



        self.conv_block = nn.Sequential(*conv_block)

    def forward(self, x):
        y = self.conv_block(x)
        return x + self.conv_block(x)

class Generator(nn.Module):
    def __init__(self, input_nc, output_nc, n_residual_blocks=10):
        super(Generator, self).__init__()

        # DOWNSAMPLING
        self.conv_1 = nn.Conv3d(in_channels=1,out_channels=64,kernel_size=(1,2,2), stride=(1,2,2))
        self.drop1 = nn.Dropout(0.3)

        # state_size (32,2,32,42)
        self.conv_2 = nn.Conv3d(in_channels=64,out_channels=128,kernel_size=(1,2,2), stride=(1,2,2))
        self.drop2 = nn.Dropout(0.3)

        # state_size (64,2,16,21)

        # RESIDUAL BLOCKS
        res_blocks = []
        for _ in range(n_residual_blocks):
            res_blocks += [ResidualBlock(128)]

        
        self.res_blocks = nn.Sequential(*res_blocks)

      
          

        # UPSAMPLING
        # out_features = in_features//2
        # for _ in range(2):
        #     model += [  nn.ConvTranspose2d(in_features, out_features, 3, stride=2, padding=1, output_padding=1),
        #                 nn.InstanceNorm2d(out_features),
        #                 nn.ReLU(inplace=True) ]
        #     in_features = out_features
        #     out_features = in_features//2

        # # Output layer
        # model += [  nn.ReflectionPad2d(3),
        #             nn.Conv2d(64, output_nc, 7),
        #             nn.Tanh() ]

        # self.model = nn.Sequential(*model)


        #  # state size : (deepth*8, 1, 4, 4)
        # self.convt1 = nn.ConvTranspose3d(in_features*8, in_features*4, (1,4,4), (1,2,2), (0,1,1), bias=False)
        # self.batchnorm1 = nn.BatchNorm3d(in_features*4)

        # #state size : (deepth*4,1, 8, 8)
        # self.convt2 = nn.ConvTranspose3d(in_features*4, in_features*2, (1,4,2),(1,2,3), (0,1,1),bias=False)
        # self.batchnorm2 = nn.BatchNorm3d(in_features*2)

        #state size : (deepth*2,2, 16, 21)
        self.convt1 = nn.ConvTranspose3d(128, 32, (1,4,4),(1,2,2), (0,1,1),bias=False)
        self.batchnorm1 = nn.BatchNorm3d(32)

        #state size : (deepth,2, 32, 42)
        self.convt2 = nn.ConvTranspose3d(32,1, (1,4,4),(1,2,2), (0,1,1),bias=False)


    def forward(self, x):
        x = self.conv_1(x)
        x = self.drop1(x)
        x = F.relu(x)
        x = self.conv_2(x)
        x = self.drop2(x)
        x = F.relu(x)
        x = self.res_blocks(x)
        x = self.convt1(x)
        x = self.batchnorm1(x)
        x = F.leaky_relu(x)
        x = self.convt2(x)
        x = F.sigmoid(x)

        return x

# class Discriminator(nn.Module):
#     def __init__(self, input_nc):
#         super(Discriminator, self).__init__()

#         # A bunch of convolutions one after another
#         model = [   nn.Conv3d(input_nc, 64, 4, stride=2, padding=1),
#                     nn.LeakyReLU(0.2, inplace=True) ]

#         model += [  nn.Conv3d(64, 128, 4, stride=2, padding=1),
#                     nn.InstanceNorm3d(128), 
#                     nn.LeakyReLU(0.2, inplace=True) ]

#         model += [  nn.Conv3d(128, 256, 4, stride=2, padding=1),
#                     nn.InstanceNorm3d(256), 
#                     nn.LeakyReLU(0.2, inplace=True) ]

#         model += [  nn.Conv3d(256, 512, 4, padding=1),
#                     nn.InstanceNorm3d(512), 
#                     nn.LeakyReLU(0.2, inplace=True) ]

#         # FCN classification layer
#         model += [nn.Conv3d(512, 1, 4, padding=1)]

#         self.model = nn.Sequential(*model)

#     def forward(self, x):
#         x =  self.model(x)
#         # Average pooling and flatten
#         return F.avg_pool2d(x, x.size()[2:]).view(x.size()[0], -1)



class Discriminator(nn.Module):
    def __init__(self, input_nc):
        super(Discriminator, self).__init__()

        # A bunch of convolutions one after another
        self.conv_1 = nn.Conv3d(in_channels=1,out_channels=32,kernel_size=(1,2,2), stride=(1,2,2))
        self.drop1 = nn.Dropout(0.3)

        # state_size (32,2,32,42)
        self.conv_2 = nn.Conv3d(in_channels=32,out_channels=64,kernel_size=(1,2,2), stride=(1,2,2))
        self.drop2 = nn.Dropout(0.3)

        # state_size (64,2,16,21)
        self.conv_3 = nn.Conv3d(in_channels=64,out_channels=128,kernel_size=(2,16,21), stride=(1,1,1))
        self.drop3 = nn.Dropout(0.3)

        self.fc_out = nn.Linear(128, 1)

    def forward(self, x):
        x = self.conv_1(x)
        x = self.drop1(x)
        x = F.relu(x)
        x = self.conv_2(x)
        x = self.drop2(x)
        x = F.relu(x)
        x = self.conv_3(x)
        x = self.drop3(x)
        x = F.relu(x)
        x = x.view(x.size()[0],128)
        x = F.sigmoid(self.fc_out(x))
        return x

In [8]:
def get_dist_matrix(numpy_array_data):
  dist = 0
  counter = 0
  for i in range(len(numpy_array_data)):
    matrix_i = numpy_array_data[i]
    for j in range(i+1, len(numpy_array_data)):
      matrix_j = numpy_array_data[j]
      counter+=1

      dist += np.abs(matrix_i-matrix_j).sum()
  
  return dist/counter


# Training

In [10]:
n = 2000
batch_size = 16
data_J = get_samples(style="J", n=n, random=True)
dataset_J = TensorDataset(torch.Tensor(data_J).type(torch.float), torch.Tensor(np.ones((data_J.shape[0], 1))).type(torch.int64))
loader_J = torch.utils.data.DataLoader(dataset_J, shuffle=True, batch_size=batch_size)

data_C = get_samples(style="C", n=n, random=True)
dataset_C = TensorDataset(torch.Tensor(data_C).type(torch.float), torch.Tensor(np.ones((data_C.shape[0], 1))).type(torch.int64))
loader_C = torch.utils.data.DataLoader(dataset_C, shuffle=True, batch_size=batch_size)

In [11]:
len(dataset_J)

1746

In [12]:
len(dataset_C)

1944

In [13]:
def extract(v):
    return v.data.storage().tolist()

In [ ]:
###### Definition of variables ######

# Training

nb_epochs = 100

# Networks
input_nc = 64
output_nc = 64


## A == J and B == C

netG_J2C = Generator(input_nc, output_nc).to(device)
netG_C2J = Generator(output_nc, input_nc).to(device)
netD_J = Discriminator(input_nc).to(device)
netD_C = Discriminator(output_nc).to(device)

# netG_A2B.apply(weights_init_normal)
# netG_B2A.apply(weights_init_normal)
# netD_A.apply(weights_init_normal)
# netD_B.apply(weights_init_normal)

# Lossess
criterion_GAN = torch.nn.MSELoss()
criterion_cycle = torch.nn.L1Loss()
criterion_identity = torch.nn.L1Loss()

# Optimizers & LR schedulers
optimizer_G = torch.optim.Adam(itertools.chain(netG_J2C.parameters(), netG_C2J.parameters()),
                                lr=1e-3, betas=(0.5, 0.999))
# optimizer_D_J = torch.optim.Adam(netD_J.parameters(), lr=1e-3, betas=(0.5, 0.999))
# optimizer_D_C = torch.optim.Adam(netD_C.parameters(), lr=1e-3, betas=(0.5, 0.999))
optimizer_D_J = optim.SGD(netD_J.parameters(), lr=1e-3, momentum=0.5)
optimizer_D_C = optim.SGD(netD_C.parameters(), lr=1e-3, momentum=0.5)

# lr_scheduler_G = torch.optim.lr_scheduler.LambdaLR(optimizer_G, lr_lambda=LambdaLR(opt.n_epochs, opt.epoch, opt.decay_epoch).step)
# lr_scheduler_D_A = torch.optim.lr_scheduler.LambdaLR(optimizer_D_A, lr_lambda=LambdaLR(opt.n_epochs, opt.epoch, opt.decay_epoch).step)
# lr_scheduler_D_B = torch.optim.lr_scheduler.LambdaLR(optimizer_D_B, lr_lambda=LambdaLR(opt.n_epochs, opt.epoch, opt.decay_epoch).step)

# # Inputs & targets memory allocation
# Tensor = torch.cuda.FloatTensor if opt.cuda else torch.Tensor
# input_A = Tensor(opt.batchSize, opt.input_nc, opt.size, opt.size)
# input_B = Tensor(opt.batchSize, opt.output_nc, opt.size, opt.size)


# fake_A_buffer = ReplayBuffer()
# fake_B_buffer = ReplayBuffer()

# # Dataset loader
# transforms_ = [ transforms.Resize(int(opt.size*1.12), Image.BICUBIC), 
#                 transforms.RandomCrop(opt.size), 
#                 transforms.RandomHorizontalFlip(),
#                 transforms.ToTensor(),
#                 transforms.Normalize((0.5,0.5,0.5), (0.5,0.5,0.5)) ]
# dataloader = DataLoader(ImageDataset(opt.dataroot, transforms_=transforms_, unaligned=True), 
#                         batch_size=opt.batchSize, shuffle=True, num_workers=opt.n_cpu)

# # Loss plot
# logger = Logger(opt.n_epochs, len(dataloader))
###################################


DJ_real_errors = []
DJ_fake_errors = []
DC_real_errors = []
DC_fake_errors = []
GJ2C_errors = []
GC2J_errors = []
J_id_errors = []
C_id_errors = []
J_cycle_errors = []
C_cycle_errors = []





###### Training ######
for epoch in range(nb_epochs):

    epoch_DJ_real_error = []
    epoch_DJ_fake_error = []
    epoch_DC_real_error = []
    epoch_DC_fake_error = []
    epoch_GJ2C_error = []
    epoch_GC2J_error = []
    epoch_J_id_error = []
    epoch_C_id_error = []
    epoch_J_cycle_error = []
    epoch_C_cycle_error = []
  


    for i, batch in enumerate(zip(loader_J, loader_C)):
        # Set model input

        # (-1, 64, 84, 2)
        real_J = batch[0][0]
        real_C = batch[1][0]


        real_J = real_J.permute(0,3,1,2)
        real_J = real_J.view(real_J.size()[0], 1, 2, 64, 84)
        real_J = real_J.to(device)
        real_C = real_C.permute(0,3,1,2)
        real_C = real_C.view(real_C.size()[0], 1, 2, 64, 84)
        real_C = real_C.to(device)

        if real_J.size()[0] != real_C.size()[0]:
          break

        N = real_J.size()[0]

        target_real = Variable(torch.ones([N, 1]).to(device))
        target_fake = Variable(torch.zeros([N, 1]).to(device))



        ###### Generators A2B and B2A ######
        optimizer_G.zero_grad()

        # Identity loss
        # G_J2C(C) should equal C if real C is fed
        same_C = netG_J2C(real_C)
        loss_identity_C = criterion_identity(same_C, real_C)  #*5.0
        epoch_C_id_error.append(extract(loss_identity_C)[0])

        # G_C2J(J) should equal J if real J is fed
        same_J = netG_C2J(real_J)
        loss_identity_J = criterion_identity(same_J, real_J) # *5
        epoch_J_id_error.append(extract(loss_identity_J)[0])

        # GAN loss
        fake_C = netG_J2C(real_J)
        pred_fake = netD_C(fake_C)
        loss_GAN_J2C = criterion_GAN(pred_fake, target_real)
        epoch_GJ2C_error.append(extract(loss_GAN_J2C)[0])

        fake_J = netG_C2J(real_C)
        pred_fake = netD_J(fake_J)
        loss_GAN_C2J = criterion_GAN(pred_fake, target_real)
        epoch_GC2J_error.append(extract(loss_GAN_C2J)[0])

        # Cycle loss
        recovered_J = netG_C2J(fake_C)
        loss_cycle_JCJ = criterion_cycle(recovered_J, real_J) # *10
        epoch_J_cycle_error.append(extract(loss_cycle_JCJ)[0])

        recovered_C = netG_J2C(fake_J)
        loss_cycle_CJC = criterion_cycle(recovered_C, real_C) # *10
        epoch_C_cycle_error.append(extract(loss_cycle_CJC)[0])

        # Total loss
        loss_G = loss_identity_C + loss_identity_J + loss_GAN_J2C + loss_GAN_C2J + loss_cycle_JCJ + loss_cycle_CJC

        # epoch_GAN_error.append(extract(loss_G)[0])
        loss_G.backward()
        
        optimizer_G.step()
        ###################################

        ###### Discriminator J ######
        optimizer_D_J.zero_grad()

        # Real loss
        pred_real = netD_J(real_J)
        loss_D_real = criterion_GAN(pred_real, target_real)
        epoch_DJ_real_error.append(extract(loss_D_real)[0])

        # # Fake loss
        # fake_A = fake_A_buffer.push_and_pop(fake_A)
        pred_fake = netD_J(fake_J.detach())
        loss_D_fake = criterion_GAN(pred_fake, target_fake)
        epoch_DJ_fake_error.append(extract(loss_D_fake)[0])

        # Total loss
        loss_D_J = (loss_D_real + loss_D_fake)*0.5
        loss_D_J.backward()

        optimizer_D_J.step()
        ###################################

        ###### Discriminator C ######
        optimizer_D_C.zero_grad()

        # Real loss
        pred_real = netD_C(real_C)
        loss_D_real = criterion_GAN(pred_real, target_real)
        epoch_DC_real_error.append(extract(loss_D_real)[0])

        # Fake loss
        # fake_B = fake_B_buffer.push_and_pop(fake_B)
        pred_fake = netD_C(fake_C.detach())
        loss_D_fake = criterion_GAN(pred_fake, target_fake)
        epoch_DC_fake_error.append(extract(loss_D_fake)[0])

        # Total loss
        loss_D_C = (loss_D_real + loss_D_fake)*0.5
        loss_D_C.backward()

        optimizer_D_C.step()
        ###################################

    

    mean_DJ_real_error = np.mean(epoch_DJ_real_error)
    mean_DJ_fake_error = np.mean(epoch_DJ_fake_error)
    mean_DC_real_error = np.mean(epoch_DC_real_error)
    mean_DC_fake_error = np.mean(epoch_DC_fake_error)
    mean_epoch_GJ2C_error = np.mean(epoch_GJ2C_error)
    mean_epoch_GC2J_error = np.mean(epoch_GC2J_error)
    mean_epoch_J_id_error = np.mean(epoch_J_id_error)
    mean_epoch_C_id_error = np.mean(epoch_C_id_error)
    mean_epoch_J_cycle_error = np.mean(epoch_J_cycle_error)
    mean_epoch_C_cycle_error = np.mean(epoch_C_cycle_error)

    
    DJ_real_errors.append(mean_DJ_real_error)
    DJ_fake_errors.append(mean_DJ_fake_error)
    DC_real_errors.append(mean_DC_real_error)
    DC_fake_errors.append(mean_DC_fake_error)
    GJ2C_errors.append(mean_epoch_GJ2C_error)
    GC2J_errors.append(mean_epoch_GC2J_error)
    J_id_errors.append(mean_epoch_J_id_error)
    C_id_errors.append(mean_epoch_C_id_error)
    J_cycle_errors.append(mean_epoch_J_cycle_error)
    C_cycle_errors.append(mean_epoch_C_cycle_error)


    if epoch % 1 == 0:
      print("------------------------------------------------------")
      print("epoch: {},  DJ real/fake errors : {}/{}       DC real/fake errors : {}/{}".format(epoch, mean_DJ_real_error, mean_DJ_fake_error, mean_DC_real_error, mean_DC_fake_error))
      print("\n")
      print(" Jazz : GAN/ID/RECOVERY errors : {}/{}/{}     Classique :  GAN/ID/RECOVERY errors : {}/{}/{}".format(mean_epoch_GC2J_error,mean_epoch_J_id_error, mean_epoch_J_cycle_error,  mean_epoch_GJ2C_error, mean_epoch_C_id_error, mean_epoch_C_cycle_error))
      print("------------------------------------------------------")








        # # Progress report (http://localhost:8097)
        # logger.log({'loss_G': loss_G, 'loss_G_identity': (loss_identity_A + loss_identity_B), 'loss_G_GAN': (loss_GAN_A2B + loss_GAN_B2A),
        #             'loss_G_cycle': (loss_cycle_ABA + loss_cycle_BAB), 'loss_D': (loss_D_A + loss_D_B)}, 
        #             images={'real_A': real_A, 'real_B': real_B, 'fake_A': fake_A, 'fake_B': fake_B})

    # # Update learning rates
    # lr_scheduler_G.step()
    # lr_scheduler_D_A.step()
    # lr_scheduler_D_B.step()

#     # Save models checkpoints
#     torch.save(netG_A2B.state_dict(), 'output/netG_A2B.pth')
#     torch.save(netG_B2A.state_dict(), 'output/netG_B2A.pth')
#     torch.save(netD_A.state_dict(), 'output/netD_A.pth')
#     torch.save(netD_B.state_dict(), 'output/netD_B.pth')
# ###################################

/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:1639: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")


------------------------------------------------------
epoch: 0,  DJ real/fake errors : 0.25289709701997426/0.2404083347101824       DC real/fake errors : 0.2402320866464475/0.25883036578467133


 Jazz : GAN/ID/RECOVERY errors : 0.2601343211504297/0.1511437296183831/0.1507696179895226     Classique :  GAN/ID/RECOVERY errors : 0.2416778768694729/0.13351199143660178/0.13310324475852722
------------------------------------------------------
------------------------------------------------------
epoch: 1,  DJ real/fake errors : 0.2509307335002707/0.24928687499203814       DC real/fake errors : 0.24434835511610048/0.25595595092948425


 Jazz : GAN/ID/RECOVERY errors : 0.25059778195455534/0.025597725183181805/0.025993256513541993     Classique :  GAN/ID/RECOVERY errors : 0.24410491922032942/0.027995438067191236/0.028078927479069166
------------------------------------------------------
------------------------------------------------------
epoch: 2,  DJ real/fake errors : 0.2480403806911696/

# Export to midi

In [23]:
def raw3D_to_midi(array, name, precision=0.7):
    '''
    params:
      array : size 64,84,2 that represents the midi
      name: (str) of the midi file
      precision: threshold from where we suppose that the note is played/ 
    '''    
    
    
    pm = pretty_midi.PrettyMIDI()
    # Create an Instrument instance
    program = pretty_midi.instrument_name_to_program('Cello')
    piano = pretty_midi.Instrument(program=program)

    for pitch_idx in range(84):
      for time_idx in range(64):
        is_played = array[time_idx, pitch_idx,0]
        if is_played > precision:
          # Means we played a new note
          end_gap = 1

          if not time_idx == 63:

            while array[time_idx+end_gap, pitch_idx,1] > precision and time_idx+end_gap < 63:
              end_gap += 1
          else:
            end_gap = 1
          note = pretty_midi.Note(
            velocity=100, pitch=25 + pitch_idx, start=1 / 8 * time_idx, end=(1 / 8) * (time_idx + end_gap))
          piano.notes.append(note)
          



    


    pm.instruments.append(piano)
    # Write out the MIDI data
    pm.write(f'midi_results/{name}.mid')

In [24]:
data_J_to_transfer = get_samples(style="J", n=200, random=True)
data_C_to_transfer = get_samples(style="C", n=200, random=True)




In [35]:
nb_sample = 2

numpy_jazz_sample = data_J_to_transfer[nb_sample]
jazz_sample = torch.tensor(numpy_jazz_sample).type(torch.float)
print(jazz_sample.size())
jazz_sample = jazz_sample.permute(2,0,1)
jazz_sample = jazz_sample.view(1, 1, 2, 64, 84)
jazz_sample = jazz_sample.to(device)


classique_from_jazz = netG_J2C(jazz_sample)


torch.Size([64, 84, 2])


/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:1639: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")


In [36]:
classique_from_jazz = classique_from_jazz.permute(0,1,3,4,2)
classique_from_jazz = classique_from_jazz.view(64,84,2)

In [37]:
raw3D_to_midi(numpy_jazz_sample, "original_jazz" + str(nb_sample))

In [38]:
raw3D_to_midi(classique_from_jazz, "classique_from_jazz" + str(nb_sample))

In [43]:
numpy_classique_sample = data_C_to_transfer[1]
classique_sample = torch.tensor(numpy_classique_sample).type(torch.float)
classique_sample = classique_sample.permute(2,0,1)
classique_sample = classique_sample.view(1, 1, 2, 64, 84)
classique_sample = classique_sample.to(device)


jazz_from_classique = netG_C2J(classique_sample)
jazz_from_classique = jazz_from_classique.permute(0,1,3,4,2)
jazz_from_classique = jazz_from_classique.view(64,84,2)

/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:1639: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")


In [42]:
jazz_from_classique.size()

torch.Size([1, 1, 2, 64, 84])

In [44]:
raw3D_to_midi(numpy_classique_sample, "original_classique")

In [45]:
raw3D_to_midi(jazz_from_classique, "jazz_from_classique")